In [ ]:
import sys
sys.path.append("/home/q621464/Desktop/Thesis/code/decision-transformer-thesis")
sys.path.append("/home/q621464/Desktop/Thesis/code/decision-transformer-thesis/atari")

In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
import csv
import logging
# make deterministic
from atari.mingpt.utils import set_seed
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
import math
from torch.utils.data import Dataset
from atari.mingpt.model_atari import GPT, GPTConfig
from atari.mingpt.trainer_atari import Trainer, TrainerConfig
from atari.mingpt.utils import sample
from collections import deque
import random
import torch
import pickle
import blosc
import argparse
from atari.create_dataset import create_dataset
import seaborn as sns
from matplotlib import pyplot as plt

from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# Apply dimensionality reduction here
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [ ]:
# set up logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

In [ ]:
class Config():
    def __init__(self, seed=123, context_length=30, epochs=5, model_type='reward_conditioned', num_steps=500000, num_buffers=50, env='SmartClimate', batch_size=128, log_to_wandb=False, trajectories_per_buffer=10, train_data_dir='../atari/data-for-dt/smart-climate-train-trajectories-v2.pkl', test_data_dir='../atari/data-for-dt/smart-climate-test-trajectories-v2.pkl') -> None:
        self.seed = seed
        self.context_length = context_length
        self.epochs = epochs
        self.model_type = model_type
        self.num_steps =num_steps
        self.num_buffers = num_buffers
        self.env = env
        self.batch_size = batch_size
        self.log_to_wandb = log_to_wandb
        self.trajectories_per_buffer = trajectories_per_buffer
        self.train_data_dir = train_data_dir
        self.test_data_dir = test_data_dir
        self.dim_reductor = None

In [ ]:
args = Config(env='SmartClimate', epochs=30, seed=123, train_data_dir='../atari/data-for-dt/smart-climate-train-trajectories-v2.pkl')
set_seed(args.seed)

In [ ]:
def reduce_dimensionality(obss):
    X_train = obss
    # Data normalizationd
    scaler = StandardScaler()
    # Fit and transform the data to perform z-score normalization
    X_train_normalized = scaler.fit_transform(X_train)

    # Initialize PCA with the desired number of components (e.g., None for all components)
    pca = PCA(n_components=None)

    # Fit and transform the training data using PCA
    X_train_pca = pca.fit_transform(X_train_normalized)

    # Get the explained variance ratio
    explained_variance_ratio = pca.explained_variance_ratio_

    # Get the cumulative explained variance ratio
    cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

    # Set a threshold for the cumulative explained variance ratio (e.g., 95% or 99%)
    threshold_variance = 0.95

    # Determine the number of components required to achieve the threshold
    n_components = np.argmax(cumulative_variance_ratio >= threshold_variance) + 1
    print(f"Number of selected components: {n_components}")

    # Reduce the dimensionality to the selected number of components
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train_normalized)
    print(f"Shape of the reeduced dimensionality dataset: {X_train_pca.shape}")

    obss_reduced = X_train_pca
    return obss_reduced, pca
    # X_test_pca = pca.transform(X_test)

In [ ]:
def create_dataset(data_dir, total_trajectories=8000):
    with open(data_dir, 'rb') as f:
        trajectories = pickle.load(f)[0:total_trajectories]
    obss = []
    actions = []
    returns = [0]
    done_idxs = []
    stepwise_returns = []    
    for traj in trajectories:
        obss += traj['observations'].tolist()
        actions += traj['actions'].tolist()
        stepwise_returns += traj['rewards'].tolist()
        done_idxs += [len(obss)]
        returns += [0]

    actions = np.array(actions)
    returns = np.array(returns)
    stepwise_returns = np.array(stepwise_returns)
    done_idxs = np.array(done_idxs)

    # -- create reward-to-go dataset
    start_index = 0
    rtg = np.zeros_like(stepwise_returns)
    for i in done_idxs:
        i = int(i)
        curr_traj_returns = stepwise_returns[start_index:i]
        for j in range(i-1, start_index-1, -1): # start from i-1
            rtg_j = curr_traj_returns[j-start_index:i-start_index]
            rtg[j] = sum(rtg_j)
        start_index = i
    print('max rtg is %d' % max(rtg))

    # -- create timestep dataset
    start_index = 0
    timesteps = np.zeros(len(actions)+1, dtype=int)
    print(f"total done idx: {len(done_idxs)}")
    for i in done_idxs:
        # print(f"done_idx: {i}")
        i = int(i)
        # print(f"start_idx: {start_index}, i: {i}")
        timesteps[start_index:i+1] = np.arange(i+1 - start_index)
        start_index = i+1
    # print('max timestep is %d' % max(timesteps))
    
    # if apply_dim_reduction:
    #     obss, dim_reductor = reduce_dimensionality(obss)
    


    return obss, actions, returns, done_idxs, rtg, timesteps

In [ ]:
class StateActionReturnDataset(Dataset):

    def __init__(self, data, block_size, actions, done_idxs, rtgs, timesteps, mean=0, std=1):
        self.block_size = block_size
        self.vocab_size = max(actions) + 1 # TODO: needs to be changed. Does it change dynamically based on the sampled data?
        self.data = data
        self.actions = actions
        self.done_idxs = done_idxs
        self.rtgs = rtgs
        self.timesteps = timesteps
        self.mean = mean
        self.std = std
    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        block_size = self.block_size // 3
        states = torch.tensor(np.array(self.data[idx:idx+block_size]), dtype=torch.float32).reshape(block_size, -1) # 
        # (block_size, 4*84*84)
        
        # print(f"There are nan values in the dataloader's batch: {torch.isnan(states).any()}")
        # mean = torch.mean(states)
        # std = torch.std(states)
        states = (states - self.mean) / self.std
        # print(f"mean: {mean}, std: {std} of the batch\n")
        # states = states / 255.
        # print(f"There are nan values in the dataloader's batch after normalization: {torch.isnan(states).any()}")
        actions = torch.tensor(self.actions[idx:idx+block_size], dtype=torch.long).unsqueeze(1) # (block_size, 1)
        rtgs = torch.tensor(self.rtgs[idx:idx+block_size], dtype=torch.float32).unsqueeze(1)
        timesteps = torch.tensor(self.timesteps[idx:idx+1], dtype=torch.int64).unsqueeze(1)
        # print(f"timesteps shape: {timesteps.shape}")

        return states, actions, rtgs, timesteps

In [ ]:
def reduce_dimensionality(obss):
    X_train = obss
    # Data normalizationd
    scaler = StandardScaler()
    # Fit and transform the data to perform z-score normalization
    X_train_normalized = scaler.fit_transform(X_train)

    # Initialize PCA with the desired number of components (e.g., None for all components)
    pca = PCA(n_components=None)

    # Fit and transform the training data using PCA
    X_train_pca = pca.fit_transform(X_train_normalized)

    # Get the explained variance ratio
    explained_variance_ratio = pca.explained_variance_ratio_

    # Get the cumulative explained variance ratio
    cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

    # Set a threshold for the cumulative explained variance ratio (e.g., 95% or 99%)
    threshold_variance = 0.95

    # Determine the number of components required to achieve the threshold
    n_components = np.argmax(cumulative_variance_ratio >= threshold_variance) + 1
    print(f"Number of selected components: {n_components}")

    # Reduce the dimensionality to the selected number of components
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train_normalized)
    print(f"Shape of the reeduced dimensionality dataset: {X_train_pca.shape}")

    obss_reduced = X_train_pca
    return obss_reduced, pca
    # X_test_pca = pca.transform(X_test)

In [ ]:
# n_head = trial.suggest_int('n_layer', 4, 8, step=2)
# context_length = 30
model_type = "reward_conditioned"
batch_size = 128
seed = 123
env = "SmartClimate"
# Load the val_dataset
val_data_dir = "../atari/data-for-dt/smart-climate-val-trajectories-v2.pkl"
# Create the dataset first
obss, actions, returns, done_idxs, rtgs, timesteps = create_dataset(val_data_dir, total_trajectories=300)

# Split your data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(obss, actions, test_size=0.1, random_state=123)
X_train, dim_reductor = reduce_dimensionality(X_train)
X_val = dim_reductor.transform(X_val)
print(f"shape of X_train: {X_train.shape} and X_val: {X_val.shape}")

In [ ]:
reduced_dataset_actions = sorted([action for action in actions])
reduced_dataset_actions_str = [str(action) for action in reduced_dataset_actions]
sns.histplot(reduced_dataset_actions_str)

In [ ]:
full_dataset_actions = sorted([action for action in actions])
full_dataset_actions_str = [str(action) for action in full_dataset_actions]
sns.histplot(full_dataset_actions_str)

In [63]:
from collections import Counter

def kl_divergence(p, q):
    """
    Calculate the KL divergence between two probability distributions p and q.
    KL(p || q) = sum(p(x) * log(p(x) / q(x)))
    """
    print(p.shape, q.shape)
    return np.sum(p * np.log(p / q))

# Count occurrences of actions in each list
counts_full_dataset = Counter(full_dataset_actions)
counts_reduced_dataset = Counter(reduced_dataset_actions)

# Get unique actions from both lists
unique_actions = sorted(set(full_dataset_actions + reduced_dataset_actions))

# Convert counts to probabilities by normalizing
total_full_dataset = len(full_dataset_actions)
total_reduced_dataset = len(reduced_dataset_actions)
p = np.array([counts_full_dataset[action] / total_full_dataset for action in unique_actions])
q = np.array([counts_reduced_dataset[action] / total_reduced_dataset for action in unique_actions])



In [71]:
# Calculate KL divergence
p = torch.rand(128, 30, 1)
q = torch.rand(128, 30, 1)
# p = p.squeeze(2).view(-1)
# q = q.squeeze(2).view(-1)
from scipy.stats import entropy
# Calculate KL divergence
kl_divergence = entropy(p, q)

print("KL Divergence:", kl_divergence)

KL Divergence: 0.4901232


In [60]:
p.squeeze(2).view(-1).shape

torch.Size([3840])

### Hyperparameter Tuning

In [ ]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
# from your_pytorch_model import YourModel  # Import your PyTorch model

def objective(trial):
    # Sample hyperparameters to be tuned by Optuna
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)
    num_hidden_units = trial.suggest_int('num_hidden_units', 32, 256, step=8)
    context_length = trial.suggest_int('context_length', 10, 30, step=10)
    n_layer = trial.suggest_int('n_layer', 4, 8, step=2)
    # n_head = trial.suggest_int('n_head', 4, 8, step=2)

    train_dataset = StateActionReturnDataset(X_train, context_length*3, y_train, done_idxs, rtgs, timesteps)
    val_dataset = StateActionReturnDataset(X_val, context_length*3, y_val, done_idxs, rtgs, timesteps)
    print(f"vocab size: {train_dataset.vocab_size}")
    
    print(len(train_dataset), len(val_dataset))
    # Define the model here
    mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size, n_layer=n_layer, n_head=8, n_embd=num_hidden_units, model_type=model_type, max_timestep=max(timesteps), input_dim=X_train.shape[1])
    model = GPT(mconf)

    # Train the model for a fixed number of epochs
    epochs = 10
    tconf = TrainerConfig(max_epochs=epochs, batch_size=batch_size, learning_rate=learning_rate, lr_decay=True, warmup_tokens=512*20, final_tokens=2*len(train_dataset)*context_length*3, num_workers=4, seed=seed, model_type=model_type, env=env, max_timestep=max(timesteps))

    rtg = 100
    max_ep_len = 100
    trainer = Trainer(model, train_dataset, None, val_dataset, tconf, env, rtg=rtg, max_ep_len=max_ep_len, num_eval_episodes=1, dim_reductor=dim_reductor)
    
    trainer.train()
    val_loss, val_accuracy = trainer.evaluate_model()
    print(f"Validation accuracy: {val_accuracy}")
    
    # Return the validation loss as the objective value to be minimized
    return val_accuracy


In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial), n_trials=50)

In [17]:
best_params = study.best_params
print(best_params)
# learning_rate = best_params['learning_rate']
# num_hidden_units = best_params['num_hidden_units']
# learning_rate, num_hidden_units
# # Train your final model using the best hyperparameters
# final_model = model(num_hidden_units)
# optimizer = optim.Adam(final_model.parameters(), lr=learning_rate)

# Train the model on the entire training data
# ...


{'learning_rate': 0.0003999625589556598, 'num_hidden_units': 208, 'context_length': 10, 'n_layer': 8}


In [22]:
fig = optuna.visualization.plot_optimization_history(study)
fig.show()

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [23]:
trial = study.best_trial
print("Best Score: ", trial.value)
print("Best Params: ")
for key, value in trial.params.items():
    print("  {}: {}".format(key, value))

Best Score:  11.0
Best Params: 
  learning_rate: 0.0003999625589556598
  num_hidden_units: 208
  context_length: 10
  n_layer: 8


In [29]:
optuna.visualization.plot_optimization_history(study)

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [28]:
study.best_trials

[FrozenTrial(number=4, state=TrialState.COMPLETE, values=[11.0], datetime_start=datetime.datetime(2023, 7, 30, 22, 24, 28, 287830), datetime_complete=datetime.datetime(2023, 7, 30, 22, 30, 51, 417171), params={'learning_rate': 0.0003999625589556598, 'num_hidden_units': 208, 'context_length': 10, 'n_layer': 8}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.001, log=True, low=1e-05, step=None), 'num_hidden_units': IntDistribution(high=256, log=False, low=32, step=8), 'context_length': IntDistribution(high=30, log=False, low=10, step=10), 'n_layer': IntDistribution(high=8, log=False, low=4, step=2)}, trial_id=4, value=None),
 FrozenTrial(number=5, state=TrialState.COMPLETE, values=[11.0], datetime_start=datetime.datetime(2023, 7, 30, 22, 30, 51, 419067), datetime_complete=datetime.datetime(2023, 7, 30, 22, 32, 38, 470879), params={'learning_rate': 0.0009556379222695591, 'num_hidden_units': 32, 'context_length': 10, 'n_laye